proctex.py is a script designed to generate document and equation objects, which it then automatically exports to .pkl (document) and .json (document & equation) objects.

Here's a quick example of how the latexmlmath function works. We're just going to pass it the fraction:
$\frac{3}{4}$.

In [21]:
import subprocess
from subprocess import PIPE
proc = subprocess.Popen(["latexmlmath", "--quiet", "-"], stderr = PIPE, stdout = PIPE, stdin = PIPE)
intext = "\\frac{3}{4}"
stdout, stderr = proc.communicate(intext)
print(stdout)

<?xml version="1.0" encoding="UTF-8"?>
<math xmlns="http://www.w3.org/1998/Math/MathML" alttext="\frac{3}{4}" display="block">
  <mfrac>
    <mn>3</mn>
    <mn>4</mn>
  </mfrac>
</math>



latexmlmath can output both contentML and displayML, the latter of which has relatively substantial coverage. There is, however, a substantial downside to latexmlmath, as one can see by running 'time latexmlmath "\\frac{3}{4}"':

<?xml version="1.0" encoding="UTF-8"?>
<math xmlns="http://www.w3.org/1998/Math/MathML" alttext="\frac{3}{4}" display="block">
  <mfrac>
    <mn>3</mn>
    <mn>4</mn>
  </mfrac>
</math>

real	0m0.632s
user	0m0.572s
sys	    0m0.048s

It takes a whole 0.6 seconds for latexmlmath to parse in a single equation. Part of this is due to the overhead of the command initializing/loading in libraries each time it is called.

In [29]:
intext = "\\begin{document} $\\frac{3}{4}$ $\\frac{5}{6}$ $\\frac{7}{8}$ \\end{document}"

proc = subprocess.Popen(["latexml", "--quiet", "-"], stderr = PIPE, stdout = PIPE, stdin = PIPE)
stdout, stderr = proc.communicate(intext)
proc = subprocess.Popen(["latexmlpost", "--pmml", "--format=xhtml" , "-"], stderr = PIPE, stdout = PIPE, stdin = PIPE)
stdout2, stderr = proc.communicate(stdout)
print(stdout2)

<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.1 plus MathML 2.0//EN" "http://www.w3.org/Math/DTD/mathml2/xhtml-math11-f.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>Untitled Document</title>
<!--Generated on Thu Jul 28 10:54:18 2016 by LaTeXML (version 0.8.2) http://dlmf.nist.gov/LaTeXML/.-->

<meta http-equiv="Content-Type" content="application/xhtml+xml; charset=UTF-8"/>
<link rel="stylesheet" href="/home/jay/hopper/hoptex/LaTeXML.css" type="text/css"/>
</head>
<body>
<div class="ltx_page_main">
<div class="ltx_page_content">
<div class="ltx_document">
<div id="p1" class="ltx_para">
<p class="ltx_p"><math xmlns="http://www.w3.org/1998/Math/MathML" id="p1.m1" class="ltx_Math" alttext="\frac{3}{4}" display="inline"><mfrac><mn>3</mn><mn>4</mn></mfrac></math> <math xmlns="http://www.w3.org/1998/Math/MathML" id="p1.m2" class="ltx_Math" alttext="\frac{5}{6}" display="inline"><mfrac><mn>5</mn><mn>6</mn></mfrac></math> <math xmlns="http: